In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def brusselator_ode(t, u, a, b):
    x, y = u
    dxdt = a - (b + 1) * x + x**2 * y
    dydt = b * x - x**2 * y
    return [dxdt, dydt]


params = [{"a": 2.0, "b": 1}, {"a": 2.0, "b": 3}, {"a": 2.0, "b": 5}]

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
for i, (param) in enumerate(params):
    a, b = param["a"], param["b"]
    # Решение системы
    t_span = [0, 50]
    t_eval = np.linspace(0, 50, 1000)
    sol = solve_ivp(
        brusselator_ode, t_span, [0.5, 0.5], args=(a, b), t_eval=t_eval, method="RK45"
    )
    # Фазовый портрет
    axes[0, i].plot(sol.y[0], sol.y[1], "b-", linewidth=1)
    axes[0, i].plot(a, b / a, "ro", markersize=8)  # Равновесие
    axes[0, i].set_xlabel("x")
    axes[0, i].set_ylabel("y")
    axes[0, i].set_title(f"a={a}, b={b}")
    # axes[0, i].grid(True)
    # Временные ряды
    axes[1, i].plot(sol.t, sol.y[0], "r-", label="x(t)")
    axes[1, i].plot(sol.t, sol.y[1], "b-", label="y(t)")
    axes[1, i].set_xlabel("Время")
    axes[1, i].set_ylabel("Концентрация")
    axes[1, i].legend()
    # axes[1, i].grid(True)
plt.tight_layout()
plt.show()

In [ ]:
params = [{"a": 1.0, "b": 0.5}, {"a": 1.0, "b": 2}, {"a": 1.0, "b": 3}]

# Параметры для модели с диффузией
Dx = 0.1
Dy = 0.05
L = 20.0
N = 100
dx = L / (N - 1)
# Начальные условия с небольшим пространственным возмущением
x0 = a + 0.1 * np.random.randn(N)
y0 = b / a + 0.1 * np.random.randn(N)


# Функция для модели с диффузией
def brusselator_diffusion(t, state):
    x = state[:N]
    y = state[N:]
    dxdt = np.zeros_like(x)
    dydt = np.zeros_like(y)
    for i in range(1, N - 1):
        dxdt[i] = (
            a
            - (b + 1) * x[i]
            + x[i] ** 2 * y[i]
            + Dx * (x[i + 1] - 2 * x[i] + x[i - 1]) / dx**2
        )
        dydt[i] = (
            b * x[i] - x[i] ** 2 * y[i] + Dy * (y[i + 1] - 2 * y[i] + y[i - 1]) / dx**2
        )
    dxdt[0] = a - (b + 1) * x[0] + x[0] ** 2 * y[0] + Dx * 2 * (x[1] - x[0]) / dx**2
    dxdt[-1] = (
        a - (b + 1) * x[-1] + x[-1] ** 2 * y[-1] + Dx * 2 * (x[-2] - x[-1]) / dx**2
    )
    dydt[0] = b * x[0] - x[0] ** 2 * y[0] + Dy * 2 * (y[1] - y[0]) / dx**2
    dydt[-1] = b * x[-1] - x[-1] ** 2 * y[-1] + Dy * 2 * (y[-2] - y[-1]) / dx**2
    return np.concatenate([dxdt, dydt])


# Решение системы с диффузией
initial_state_diff = np.concatenate([x0, y0])
t_span_diff = (0, 50)
t_eval_diff = np.linspace(t_span_diff[0], t_span_diff[1], 200)
solution_diff = solve_ivp(
    brusselator_diffusion,
    t_span_diff,
    initial_state_diff,
    t_eval=t_eval_diff,
    method="BDF",
    rtol=1e-6,
)
# Визуализация пространственно-временной динамики
x_space_time = solution_diff.y[:N, :]
y_space_time = solution_diff.y[N:, :]
space = np.linspace(0, L, N)
time = solution_diff.t
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
# Пространственно-временная диаграмма для x
im1 = axes[0, 0].imshow(x_space_time, aspect="auto", extent=[0, time[-1], L, 0])
axes[0, 0].set_xlabel("t")
axes[0, 0].set_ylabel("r")
axes[0, 0].set_title("Динамика x(r,t)")
plt.colorbar(im1, ax=axes[0, 0])
# Пространственно-временная диаграмма для y
im2 = axes[0, 1].imshow(y_space_time, aspect="auto", extent=[0, time[-1], L, 0])
axes[0, 1].set_xlabel("t")
axes[0, 1].set_ylabel("r")
axes[0, 1].set_title("Динамика y(r,t)")
plt.colorbar(im2, ax=axes[0, 1])
# Профили концентраций в разные моменты времени
axes[1, 0].plot(space, x_space_time[:, 0], "b-", label="t=0", alpha=0.7)
axes[1, 0].plot(space, x_space_time[:, 20], "g-", label=f"t={time[20]:.1f}", alpha=0.7)
axes[1, 0].plot(space, x_space_time[:, 30], "r-", label=f"t={time[30]:.1f}", alpha=0.7)
axes[1, 0].plot(space, x_space_time[:, 40], "k-", label=f"t={time[40]:.1f}", alpha=0.7)
axes[1, 0].set_xlabel("r")
axes[1, 0].set_ylabel("Концентрация x")
axes[1, 0].set_title("Профили x")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)
# Осцилляции в одной точке пространства
point_idx = 10
axes[1, 1].plot(time, x_space_time[point_idx, :], "r-", label="x(t) в центре")
axes[1, 1].plot(time, y_space_time[point_idx, :], "b-", label="y(t) в центре")
axes[1, 1].set_xlabel("t")
axes[1, 1].set_ylabel("Концентрация")
axes[1, 1].set_title(f"При r={space[point_idx]:.1f}")
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()